# Init

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Smartech/MinaRumas')
sys.path.append('/content/drive/MyDrive/Smartech/MinaRumas/libs')

import os
os.chdir('/content/drive/MyDrive/Smartech/MinaRumas')

# Install paquetes python

## install

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

## Import

In [ ]:
import os
import cv2
import time
import json
import base64
import torch
import requests
import numpy as np
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict, deque

#Import librerias propias
from monitor.ruma_monitor import RumaMonitor
from utils.paths import generar_ruta_salida

# Ruma logic

## import

## Logic class

# Process video

In [ ]:
def process_video(video_path, output_path, start_time_sec, end_time_sec,
                 model_det_path, model_seg_path, detection_zone, camera_number):
    """
    Procesa un video completo usando el monitor de rumas

    Args:
        video_path: Ruta del video de entrada
        output_path: Ruta del video de salida
        start_time_sec: Tiempo de inicio en segundos
        end_time_sec: Tiempo de fin en segundos
        model_det_path: Ruta del modelo de detección
        model_seg_path: Ruta del modelo de segmentación
        detection_zone: Zona de detección como array numpy
        camera_number: Número de cámara (1, 2, o 3)
    """

    # Mapear número de cámara a serial
    camera_serials = {
        1: "DS-7104NI-Q1-1",
        2: "DS-7104NI-Q1-2",
        3: "DS-7104NI-Q1-3"
    }

    camera_sn = camera_serials.get(camera_number, "DS-7104NI-Q1-1")

    # Inicializar monitor
    monitor = RumaMonitor(model_det_path, model_seg_path, detection_zone, camera_sn)

    # Configurar video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video: {width}x{height} @ {fps} FPS")

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    start_frame = int(start_time_sec * fps)
    end_frame = int(end_time_sec * fps)

    print(f"Procesando frames {start_frame} a {end_frame}")

    frame_count = 0

    with torch.no_grad():
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count > end_frame:
                break

            if frame_count >= start_frame:
                # Procesar frame
                processed_frame = monitor.process_frame(frame, frame_count, fps)
                out.write(processed_frame)

                # Progreso
                if frame_count % 50 == 0:
                    print(f"Procesados {frame_count} frames")
                    print(f"Rumas activas: {sum(1 for r in monitor.rumas.values() if r.is_active)}")

            frame_count += 1

    cap.release()
    out.release()
    print(f"Procesamiento completado. Video guardado en {output_path}")
    print(f"Total de rumas detectadas: {len(monitor.rumas)}")

# Uso de lógica

In [ ]:
# Process video, no --
if __name__ == "__main__":
    video_path = 'video/video_cam3_0422.mp4'
    output_path = generar_ruta_salida(video_path)
    start_time_sec = 0# 10
    end_time_sec =120 # 100

    model_det_path = 'models/model_detection.pt'
    model_seg_path = 'models/model_segmentation.pt'

    camera_number = 3  # Cambiar según la cámara

    if camera_number ==1: ## donde la ruma estan esparcidas en un campo
      detection_zone = np.array([
          [20, 691], [18, 779], [414, 881], [709, 759],
          [1675, 1060], [1902, 890], [1902, 667], [704, 416]
          ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)
    elif camera_number ==2: # se puede ver la escalera o faja
      detection_zone = np.array([
          [1740,533],[1788,1075],[741,1074],[433,591],[400,480],[536,442],[1008,365],[1061,398],[1128,389],[1214,431],[1369,397]
          ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)
    elif camera_number ==3:
      detection_zone = np.array([
          [1862,767],[1443,633],[794,1077],[123,1072],[3,711],[4,517],[1122,256],[1893,450],[1860,534],[1873,611],[1855,689]
          ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)


    process_video(
        video_path, output_path, start_time_sec, end_time_sec,
        model_det_path, model_seg_path, detection_zone, camera_number
    )
